<a href="https://colab.research.google.com/github/huytuan3108/btap_AI/blob/main/ID_card.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install easyocr
import easyocr
import pandas as pd
import re
from google.colab import files

In [13]:
import easyocr
import pandas as pd
import re
from google.colab import files

def fix_invalid_date(date_str):
    try:
        day, month, year = map(int, date_str.split('/'))
        if day > 31: day = 31
        if month > 12: month = 12
        return f"{day:02}/{month:02}/{year}"
    except:
        return date_str

def clean_ocr_text(text):
    return text.replace('\x0c', ' ').strip()

def extract_fields(text):
    text = clean_ocr_text(text)
    info = {}

    # Số ID
    id_match = re.search(r'\b(\d{12})\b', text)
    if id_match:
        info['Số ID'] = id_match.group(1)

    # Họ tên
    name_match = re.search(r'(?:Họ và tên|Full name)\s*[:\s]*(.*?)(?=\n|$)', text, re.IGNORECASE | re.DOTALL)
    if name_match:
        name = name_match.group(1).strip()
        name = re.sub(r'^\s*/\s*Full name\s*:\s*', '', name, flags=re.IGNORECASE)
        if re.search(r'[A-ZÀÁẠẢÃÂĂĐÈÉẸẺẼÊÌÍỊỈĨÒÓỌỎÕÔƠÙÚỤỦŨƯỲÝỴỶỸ]', name):
            info['Họ tên'] = name
    else:
        all_caps_lines = [line.strip() for line in text.split('\n')
                         if re.fullmatch(r'[A-ZÀÁẠẢÃÂĂĐÈÉẸẺẼÊÌÍỊỈĨÒÓỌỎÕÔƠÙÚỤỦŨƯỲÝỴỶỸ\s]{5,}', line) and len(line.split()) >= 2]
        if all_caps_lines:
            info['Họ tên'] = all_caps_lines[0]

    # Ngày sinh
    dob_match = re.search(r'(\d{1,2}/\d{1,2}/\d{4})', text)
    if dob_match:
        info['Ngày sinh'] = fix_invalid_date(dob_match.group(1))

    # Giới tính
    gender_match = re.search(r'(?:Giới tính|Sex)\s*[:\s]*(Nam|Nữ)', text, re.IGNORECASE)
    if gender_match:
        info['Giới tính'] = gender_match.group(1).capitalize()

    # Quê quán
    origin_match = re.search(r'(?:Quê quán|Place of origin)\s*[:\s]*(.*?)(?=\n(?:Nơi thường trú|Place of residence|$))', text, re.DOTALL | re.IGNORECASE)
    if origin_match:
        origin = origin_match.group(1).strip()
        origin = re.sub(r'\s*/\s*.*', '', origin)
        info['Quê quán'] = origin.strip()

    # Nơi thường trú
    residence_match = re.search(r'(?:Nơi thường trú|Place of residence)\s*[:\s]*(.*)', text, re.DOTALL | re.IGNORECASE)
    if residence_match:
        residence = residence_match.group(1).strip()
        residence = re.sub(r'\s*/\s*.*', '', residence)
        info['Nơi thường trú'] = residence.strip()

    return info

# Tải ảnh lên
print("Vui lòng tải ảnh CCCD lên:")
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

# Khởi tạo EasyOCR
reader = easyocr.Reader(['vi', 'en'], gpu=True)  # Sử dụng GPU trên Colab
results = reader.readtext(image_path, detail=0, paragraph=True)
text = "\n".join(results)

print("Văn bản OCR được:")
print(text)

# Trích xuất thông tin
info = extract_fields(text)

# In thông tin trích được
print("\nThông tin trích được:")
for k, v in info.items():
    print(f"{k}: {v}")

# Lưu ra Excel
excel_path = 'cccd_output.xlsx'
df = pd.DataFrame([info])
df.to_excel(excel_path, index=False, engine='openpyxl')

# Tải file Excel về
print("\nTải file Excel về:")
files.download(excel_path)

Vui lòng tải ảnh CCCD lên:


Saving cccd.jpg to cccd (1).jpg
Văn bản OCR được:
CĂN CƯỚC CÔNG DÂN Citizen Identity Card Số / No 075206001761 Họ và tên / Full name: ĐĂNG HUY TUẨN Ngày sinh / Date of birth: 31/08/2006 Giới tính / Sex: Nam Quốc tịch / Nationality:  Việt Nam Quê quán / Place of origin: Phương Tú, Ứng Hòa, Hà Nội Nơi thường trú / Place of residence 541, Ịổ 4, Kp 12 An Bình, Thành phố Biên Hòa, Đổng Nai

Thông tin trích được:
Số ID: 075206001761
Họ tên: ĐĂNG HUY TUẨN Ngày sinh / Date of birth: 31/08/2006 Giới tính / Sex: Nam Quốc tịch / Nationality:  Việt Nam Quê quán / Place of origin: Phương Tú, Ứng Hòa, Hà Nội Nơi thường trú / Place of residence 541, Ịổ 4, Kp 12 An Bình, Thành phố Biên Hòa, Đổng Nai
Ngày sinh: 31/08/2006
Giới tính: Nam
Nơi thường trú: 

Tải file Excel về:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>